In [25]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch import nn
from skorch import NeuralNetRegressor
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDRegressor, BayesianRidge, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


# Configuration
sns.set(color_codes=True)
random_state = 4720


In [26]:
data = pd.read_csv('./data.csv', index_col=0)

# Category indexing
i_voltage = np.arange(0, 9)
i_angles = np.arange(9, 18)
i_pgen = np.arange(18, 27)
i_qgen = np.arange(27, 36)
i_pdem = np.arange(36, 45)
i_qdem = np.arange(45, 54)
i_pflow = np.arange(54, 135)
i_qflow = np.arange(135, 216)
i_risk = 216


In [27]:
class RegressorModule(nn.Module):
    def __init__(self, num_units=36, nonlin=F.relu, drop=.3):
        super(RegressorModule, self).__init__()

        self.module = nn.Sequential(
            nn.Linear(64, num_units),
            nn.LeakyReLU(),
            nn.Dropout(p=drop),
            nn.Linear(num_units, 1),
        )

    def forward(self, X):
        X = self.module(X)
        return X


In [28]:
def algorithm(X, y):
    Xtrain, Xtest, ytrain, ytest = train_test_split(
        X, y, stratify=y, test_size=0.3, random_state=4720)

    estimators = ['LinearRegression', 'SVR', 'ElasticNet', 'GradientBoostingRegressor',
                  'DecisionTreeRegressor', 'RandomForestRegressor', 'NeuralNetwork']

    results = dict()

    for estimator in estimators:
        params = {
            'scaler': ['passthrough', StandardScaler(), MinMaxScaler()],
        }
        if estimator == "LinearRegression":
            params['dimension'] = ['passthrough',
                                   PCA(random_state=random_state)]
            params['selection__k'] = [
                'all', 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
            model = LinearRegression()
        elif estimator == "SVR":
            params['dimension'] = ['passthrough',
                                   PCA(random_state=random_state)]
            params['selection__k'] = [
                'all', 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
            params['estimation__kernel'] = ['rbf']
            params['estimation__C'] = np.logspace(-2, 3, base=10)
            model = SVR()
        elif estimator == "ElasticNet":
            params['dimension'] = ['passthrough',
                                   PCA(random_state=random_state)]
            params['selection__k'] = [
                'all', 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
            params['estimation__alpha'] = np.logspace(-2, 3, base=10)
            model = ElasticNet()
        elif estimator == "GradientBoostingRegressor":
            params['dimension'] = ['passthrough',
                                   PCA(random_state=random_state)]
            params['selection__k'] = [
                'all', 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
            # params['estimation__max_depth'] = [None, np.arange(1, 50)]
            model = GradientBoostingRegressor()
        elif estimator == "DecisionTreeRegressor":
            params['dimension'] = ['passthrough',
                                   PCA(random_state=random_state)]
            params['selection__k'] = [
                'all', 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
            # params['estimation__max_depth'] = [None, np.arange(1, 50)]
            model = DecisionTreeRegressor()
        elif estimator == "RandomForestRegressor":
            params['dimension'] = ['passthrough',
                                   PCA(random_state=random_state)]
            params['selection__k'] = [
                'all', 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
            # params['estimation__max_depth'] = [None, np.arange(1, 50)]
            model = RandomForestRegressor()
        elif estimator == "NeuralNetwork":
            Xtrain = np.array(Xtrain)
            ytrain = np.array(ytrain)
            Xtest = np.array(Xtest)
            ytest = np.array(ytest)
            Xtrain = Xtrain.astype(np.float32)
            ytrain = ytrain.astype(np.float32)
            ytrain = ytrain.reshape(-1, 1)
            Xtest = Xtest.astype(np.float32)
            ytest = ytest.astype(np.float32)
            ytest = ytest.reshape(-1, 1)
            model = NeuralNetRegressor(
                RegressorModule,
                criterion=nn.MSELoss,
                max_epochs=600,
                optimizer=optim.Adam,
                optimizer__lr=.005,
                device='cuda'  # uncomment this to train with CUDA
            )

        model_pipeline = [('scaler', 'passthrough'),
                          ('dimension', 'passthrough'),
                          ('selection', SelectKBest(
                              score_func=f_regression, k='all')),
                          ('estimation', model)]
        pipeline = Pipeline(model_pipeline)

        grid = RandomizedSearchCV(
            pipeline, params, n_jobs=-1, cv=5, n_iter=50, random_state=random_state)

        if estimator == "NeuralNetwork":
            grid = GridSearchCV(
                pipeline, params, cv=2)

        grid.fit(Xtrain, ytrain)
        results[estimator] = {
            'train_score': grid.score(Xtrain, ytrain),
            'test_score': grid.score(Xtest, ytest),
            'best_params': grid.best_params_
        }

    return results
    # for estimator in estimators:
    #     print(estimator + " train/test score of is %.2f / %.2f " %
    #           (results[estimator]['train_score'], results[estimator]['test_score']))


# Dataset 1

In [29]:
data_raw = pd.concat([data.iloc[:, i_voltage], data.iloc[:, i_angles],
                      data.iloc[:, i_pgen], data.iloc[:,
                                                      i_qgen], data.iloc[:, i_pdem],
                     data.iloc[:, i_qdem], data.iloc[:, i_pflow], data.iloc[:, i_qflow], data.iloc[:, i_risk]], axis=1)
# Assumption that zero data is not important
# Remove all the full-zero columns, then delete all the rows with NA values
data_raw = data_raw.loc[:, data_raw.any()].dropna(axis=0)
# Split the features (X) from the labels (y)
X1 = data_raw.iloc[:, 0:(data_raw.shape[1]-1)]
print(X1.shape)
y1 = data_raw.iloc[:, (data_raw.shape[1]-1)]
print(y1.shape)

(4802, 64)
(4802,)


# Dataset 2

In [30]:
data_raw = pd.concat([data.iloc[:, i_voltage], data.iloc[:, i_angles],
                      data.iloc[:, i_pgen], data.iloc[:,
                                                      i_qgen], data.iloc[:, i_pdem],
                     data.iloc[:, i_qdem], data.iloc[:, i_pflow], data.iloc[:, i_qflow], data.iloc[:, i_risk]], axis=1)
# Remove empty feature columns
data_raw = data_raw.loc[:, data_raw.any()]
X2raw = data_raw.iloc[:, 0:(data_raw.shape[1]-1)]
y2 = data_raw.iloc[:, (data_raw.shape[1]-1)]
print('Missing: %d' % sum(np.isnan(X2raw.to_numpy()).flatten()))
imputer = IterativeImputer(random_state=random_state)
X2 = imputer.fit_transform(X2raw)
print('Missing: %d' % sum(np.isnan(X2).flatten()))



Missing: 153
Missing: 0


# Algorithm

In [31]:
result1 = algorithm(X1, y1)
result2 = algorithm(X2, y2)


C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1      115.2162       19.2438  0.0390
      2        9.3819        1.2824  0.0400
      3        0.9144        0.1006  0.0440
      4        0.2045        0.1059  0.0430


C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:358: RuntimeWarning: invalid value encountered in true_divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom


      5        0.1765        0.0919  0.0420
      6        0.1435        0.0855  0.0440
      7        0.1305        0.0846  0.0430
      8        0.1214        0.0810  0.0390
      9        0.1285        0.0787  0.0410
     10        0.1164        0.0740  0.0380
     11        0.1126        0.0723  0.0410
     12        0.1064        0.0647  0.0400
     13        0.1035        0.0614  0.0390
     14        0.1050        0.0552  0.0390
     15        0.0892        0.0426  0.0390
     16        0.0816        0.0366  0.0400
     17        0.0822        0.0516  0.0390
     18        0.0769        0.0353  0.0380
     19        0.0812        0.0377  0.0390
     20        0.0758        0.0342  0.0380
     21        0.0736        0.0513  0.0390
     22        0.0735        0.0346  0.0380
     23        0.0693        0.0352  0.0370
     24        0.0676        0.0369  0.0380
     25        0.0668        0.0329  0.0390
     26        0.0677        0.0308  0.0420
     27        0.0641        0.0

C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:358: RuntimeWarning: invalid value encountered in true_divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom


...

      6        0.0383        0.0357  0.0370
      7        0.0362        0.0325  0.0380
      8        0.0333        0.0299  0.0370
      9        0.0328        0.0285  0.0370
     10        0.0280        0.0265  0.0370
     11        0.0257        0.0248  0.0370
     12        0.0264        0.0238  0.0370
     13        0.0248        0.0212  0.0370
     14        0.0240        0.0222  0.0380
     15        0.0218        0.0197  0.0370
     16        0.0227        0.0187  0.0370
     17        0.0217        0.0180  0.0370
     18        0.0212        0.0172  0.0380
     19        0.0213        0.0166  0.0370
     20        0.0213        0.0154  0.0370
     21        0.0203        0.0143  0.0370
     22        0.0205        0.0150  0.0360
     23        0.0198        0.0139  0.0370
     24        0.0187        0.0144  0.0380
     25        0.0173        0.0125  0.0370
     26        0.0183        0.0124  0.0370
     27        0.0196        0.0133  0.0370
     28        0.0179        0.0

C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


      6        0.0481        0.0336  0.0370
      7        0.0432        0.0318  0.0360
      8        0.0431        0.0296  0.0440
      9        0.0394        0.0282  0.0370
     10        0.0351        0.0284  0.0590
     11        0.0343        0.0249  0.0520
     12        0.0360        0.0243  0.0530
     13        0.0341        0.0243  0.0450
     14        0.0315        0.0234  0.0410
     15        0.0290        0.0225  0.0380
     16        0.0308        0.0190  0.0390
     17        0.0264        0.0219  0.0390
     18        0.0298        0.0187  0.0370
     19        0.0265        0.0157  0.0380
     20        0.0273        0.0163  0.0380
     21        0.0249        0.0140  0.0380
     22        0.0228        0.0172  0.0380
     23        0.0237        0.0158  0.0390
     24        0.0245        0.0125  0.0380
     25        0.0224        0.0138  0.0380
     26        0.0240        0.0128  0.0380
     27        0.0209        0.0119  0.0370
     28        0.0212        0.0

C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Philippe vanMastrigt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


      3        0.0226        0.0110  0.0770
      4        0.0188        0.0094  0.0760
      5        0.0181        0.0091  0.0760
      6        0.0168        0.0093  0.0780
      7        0.0160        0.0090  0.0830
      8        0.0157        0.0089  0.0790
      9        0.0139        0.0084  0.0760
     10        0.0141        0.0075  0.0770
     11        0.0133        0.0076  0.0760
     12        0.0131        0.0078  0.0770
     13        0.0130        0.0070  0.0770
     14        0.0136        0.0080  0.0730
     15        0.0131        0.0080  0.0720
     16        0.0129        0.0067  0.0750
     17        0.0129        0.0092  0.0730
     18        0.0130        0.0072  0.0750
     19        0.0120        0.0079  0.0730
     20        0.0129        0.0092  0.0780
     21        0.0128        0.0091  0.0720
     22        0.0133        0.0068  0.0720
     23        0.0131        0.0080  0.0730
     24        0.0125        0.0076  0.0740
     25        0.0127        0.0

# Results

In [1]:
for estimator in estimators:
    print(result1[estimator]["test_score"])

NameError: name 'estimators' is not defined

In [38]:
for estimator in estimators:
    print(result2[estimator]["test_score"])

0.798672549304374
0.9630406871945255
0.6807236208417979
0.9378327396606873
0.9124246163916444
0.9588061437295368
0.9355640297464796
